`shutil.copytree` works recursively. You can just use it to `cp` your folders and content.

In [3]:
import os, shutil

def copytree(src, dst, symlinks=False, ignore=None, copy_function=shutil.copy2,
             ignore_dangling_symlinks=False):
    """Recursively copy a directory tree.

    The destination directory must not already exist.
    If exception(s) occur, an Error is raised with a list of reasons.

    If the optional symlinks flag is true, symbolic links in the
    source tree result in symbolic links in the destination tree; if
    it is false, the contents of the files pointed to by symbolic
    links are copied. If the file pointed by the symlink doesn't
    exist, an exception will be added in the list of errors raised in
    an Error exception at the end of the copy process.

    You can set the optional ignore_dangling_symlinks flag to true if you
    want to silence this exception. Notice that this has no effect on
    platforms that don't support os.symlink.

    The optional ignore argument is a callable. If given, it
    is called with the `src` parameter, which is the directory
    being visited by copytree(), and `names` which is the list of
    `src` contents, as returned by os.listdir():

        callable(src, names) -> ignored_names

    Since copytree() is called recursively, the callable will be
    called once for each directory that is copied. It returns a
    list of names relative to the `src` directory that should
    not be copied.

    The optional copy_function argument is a callable that will be used
    to copy each file. It will be called with the source path and the
    destination path as arguments. By default, copy2() is used, but any
    function that supports the same signature (like copy()) can be used.

    """
    names = os.listdir(src)
    if ignore is not None:
        ignored_names = ignore(src, names)
    else:
        ignored_names = set()
    
    if not os.path.exists(dst): os.makedirs(dst)
    else: print(f'{dst} exists.')
    errors = []
    for name in names:
        if name in ignored_names:
            continue
        srcname = os.path.join(src, name)
        dstname = os.path.join(dst, name)
        try:
            if os.path.islink(srcname):
                # if the link exists
                if name in listdir(dst):
                    print(f'{srcname} exists in {dst}')
                    continue
                linkto = os.readlink(srcname)
                if symlinks:
                    os.symlink(linkto, dstname)
                else:
                    # ignore dangling symlink if the flag is on
                    if not os.path.exists(linkto) and ignore_dangling_symlinks:
                        continue
                    # otherwise let the copy occurs. copy2 will raise an error
                    copy_function(srcname, dstname)
            elif os.path.isdir(srcname):
                copytree(srcname, dstname, symlinks, ignore, copy_function)
            else:
                # if the file exists
                if name in listdir(dst):
                    print(f'{srcname} exists in {dst}')
                    continue
                # Will raise a SpecialFileError for unsupported file types
                copy_function(srcname, dstname)
        # catch the Error from the recursive copytree so that we can
        # continue with other files
        except Error as err:
            errors.extend(err.args[0])
        except EnvironmentError as why:
            errors.append((srcname, dstname, str(why)))
    try:
        copystat(src, dst)
    except OSError as why:
        if WindowsError is not None and isinstance(why, WindowsError):
            # Copying file access times may fail on Windows
            pass
        else:
            errors.extend((src, dst, str(why)))
    if errors:
        raise Error(errors)

In [4]:
def relocate(src, dst):
    """To Relocate (Copy) the Folders and content from src folder to dst folder.
    Specific for the use case of `cp` the fiile from cloud drive to your local machine.

    @src: input folder
    @dst: output folder
    """
    copytree(
        src,
        dst,
        ignore = shutil.ignore_patterns('*.ini', '*.gdoc', '*.gsheet')
    )

relocate(
    r'C:\Users\Kevin\OneDrive - Wisdom Asset Management Ltd',
    r'D:\OneDrive',
)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'D:\\OneDrive'

In [12]:
help(relocate)

Help on function relocate in module __main__:

relocate(src, dst)
    To Relocate (Copy) the Folders and content from src folder to dst folder.
    Specific for the use case of `cp` the fiile from cloud drive to your local machine.
    
    @src: input folder
    @dst: output folder



In [8]:
import os


def check_log():
    # read the log
    if os.path.exists('log.txt'):
        print('log.txt exists.')
        pass
    # create a new log
    else:
        open('log.txt', 'x')
        print('log.txt created.')

def relocate(input_folder, output_folder):
    check_log()
    
    # DFS
    print(
        [os.path.join(input_folder, mypath) for mypath in os.listdir(input_folder)],'\n',
        len([os.path.join(input_folder, mypath) for mypath in os.listdir(input_folder)])
    )

    
relocate(
    r'C:\Users\HsiangHung-WisdomGro\OneDrive - Wisdom Asset Management Ltd',
    r'C:\Users\HsiangHung-WisdomGro\OneDrive - Wisdom Asset Management Ltd'
)

log.txt exists.
['C:\\Users\\HsiangHung-WisdomGro\\OneDrive - Wisdom Asset Management Ltd\\.849C9593-D756-4E56-8D6E-42412F2A707B', 'C:\\Users\\HsiangHung-WisdomGro\\OneDrive - Wisdom Asset Management Ltd\\151625-001_202105.pdf', 'C:\\Users\\HsiangHung-WisdomGro\\OneDrive - Wisdom Asset Management Ltd\\88627010.6000.VALUATION.2022-03-03.85244266.pdf', 'C:\\Users\\HsiangHung-WisdomGro\\OneDrive - Wisdom Asset Management Ltd\\89308221.6000.VALUATION.2022-03-03.85247704.pdf', 'C:\\Users\\HsiangHung-WisdomGro\\OneDrive - Wisdom Asset Management Ltd\\89308502.6000.VALUATION.2022-03-03.85243664.pdf', 'C:\\Users\\HsiangHung-WisdomGro\\OneDrive - Wisdom Asset Management Ltd\\accu_1_en.jpg', 'C:\\Users\\HsiangHung-WisdomGro\\OneDrive - Wisdom Asset Management Ltd\\accu_2_en.jpg', 'C:\\Users\\HsiangHung-WisdomGro\\OneDrive - Wisdom Asset Management Ltd\\Attachments', 'C:\\Users\\HsiangHung-WisdomGro\\OneDrive - Wisdom Asset Management Ltd\\AWS', 'C:\\Users\\HsiangHung-WisdomGro\\OneDrive - Wisdom

In [ ]:
import os
from os import listdir, path
# from distutils.dir_util import copy_tree
import shutil
from tqdm import tqdm

def get_directory_size(directory):
    """Returns the `directory` size in bytes."""
    total = 0
    try:
        # print("[+] Getting the size of", directory)
        for entry in os.scandir(directory):
            if entry.is_file():
                # if it's a file, use stat() function
                total += entry.stat().st_size
            elif entry.is_dir():
                # if it's a directory, recursively call this function
                total += get_directory_size(entry.path)
    except NotADirectoryError:
        # if `directory` isn't a directory, get the file size then
        return os.path.getsize(directory)
    except PermissionError:
        # if for whatever reason we can't open the folder, return 0
        return 0
    return total

def get_size_format(b, factor=1024, suffix="B"):
    """
    Scale bytes to its proper byte format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    for unit in ["", "K", "M", "G", "T", "P", "E", "Z"]:
        if b < factor:
            return f"{b:.2f}{unit}{suffix}"
        b /= factor
    return f"{b:.2f}Y{suffix}"

bigger_than_50g = []
root = r'G:\My Drive\my_drive'
new_root = r'C:\Users\HsiangHung-WisdomGro\OneDrive - Wisdom Asset Management Ltd\my_drive'

for master in tqdm( listdir(root) ):
    if master in listdir(new_root): continue
    master_path = path.join(root, master)
    master_isdir = path.isdir(master_path)
    
    if master_isdir:
        shutil.copytree(master_path, path.join(new_root, master), ignore = shutil.ignore_patterns('*.ini', '*.gdoc', '*.gsheet'))

        # print(master,
        #     get_size_format(
        #         get_directory_size(master)
        #     )
        # )
        # bigger_than_50g.append([master, get_size_format( get_directory_size(master) )])
    else:
        shutil.copy(master_path, path.join(new_root, master))

        # print(master)
        # copy_tree(r"G:\My Drive\my_drive\test", r"C:\Users\HsiangHung-WisdomGro\OneDrive - Wisdom Asset Management Ltd")

    # print( master, master_isdir )
    # while master_isdir:
    #     listdir(master, )